# Checking Expectations

In the previous chapters on [tracing](Tracer.ipynb) and [interactive debugging](Debugger.ipynb), we have seen how to observe executions. By checking our observations against our expectations, we can find out when and how the program state is faulty. So far, we have assumed that this check would be done by _humans_ – that is, us. However, having this check done by a _computer_, for instance as part of the execution, is infinitely more rigorous and efficient. In this chapter, we introduce techniques to _specify_ our expectations and to check them at runtime, enabling us to detect faults _right as they occur_.

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* You should have read [chapter on tracing executions](Tracer.ipynb).

In [ ]:
import bookutils

## Synopsis

<!-- Automatically generated. Do not edit. -->



_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
print(int_fuzzer())
```
```python
=> 76.5

```


## Introducing Assertions

[Tracers](Tracer.ipynb) and [Interactive Debuggers](Debugger.ipynb) are very flexible tools that allow you to observe precisely what happens during a program execution. It is still _you_, however, who has to check program states and traces against your expectations. There is nothing wrong with that – except that checking hundreds of statements or variables can quickly become a pretty boring and tedious task.

Processing and checking large amounts of data is actually precisely what _computers were invented for_. Hence, we should aim to _delegate such checking tasks to our computers_ as much as we can. This automates another essential part of debugging – maybe even _the_ most essential part.

### Assertions

The standard tool for having the computer check specific conditions at runtime is called an _assertion_. An assertion takes the form

```python
assert condition
```

and states that, at runtime, the computer should check that `condition` holds, e.g. evaluates to True. If the condition holds, then nothing happens:

In [ ]:
assert True

If the condition evaluates to _False_, however, then the assertion _fails_, indicating an internal error.

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError():
    assert False

A common usage for assertions is for _testing_. For instance, we can test a square root function as

In [ ]:
def test_square_root():
    assert square_root(4) == 2
    assert square_root(9) == 3
    ...

and `test_square_root()` will fail if `square_root()` returns a wrong value.

Assertions are available in all programming languages. You can even go and implement assertions yourself:

In [ ]:
def my_own_assert(cond):
    if not cond:
        raise AssertionError

... and get (almost) the same functionality:

In [ ]:
with ExpectError():
    my_own_assert(2 + 2 == 5)

### Assertion Diagnostics

In most languages, _built-in assertions_ offer a bit more functionality than what can be obtained with self-defined functions. Most notably, built-in assertions 

* frequently tell _which condition_ failed (`2 + 2 == 5`)
* frequently tell _where_ the assertion failed (`line 2`), and
* are _optional_ – that is, they can be turned off to save computation time.

C and C++, for instance, provide an `assert()` function that does all this:

In [ ]:
# ignore
open('testassert.c', 'w').write(r'''
#include "assert.h"

int main(int argc, char *argv[]) {
    assert(2 + 2 == 5);
    printf("Foo\n");
}
''');

In [ ]:
# ignore
from bookutils import print_content

In [ ]:
print_content(open('testassert.c').read())

If we compile this function and execute it, the assertion (epectedly) fails:

In [ ]:
!cc -o testassert testassert.c

In [ ]:
!./testassert

How would the C `assert()` function be able to report the condition and the current location? In fact, `assert()` is commonly implemented as a _macro_ that besides checking the condition, also turns it into a _string_ for a potential error message. Additional macros such as `__FILE__` and `__LINE__` expand into the current location and line, which can then all be used in the assertion error message.

A very simple definition of `assert()` that provides the above diagnostics looks like this:

In [ ]:
# ignore
open('assert.h', 'w').write(r'''
#include <stdio.h>
#include <stdlib.h>

#ifndef NDEBUG
#define assert(cond) \
    if (!(cond)) { \
        fprintf(stderr, "Assertion failed: %s, function %s, file %s, line %d", \
            #cond, __func__, __FILE__, __LINE__); \
        exit(1); \
    }
#else
#define assert(cond) ((void) 0)
#endif
''');

In [ ]:
print_content(open('assert.h').read())

(If you think that this is cryptic, you should have a look at an [_actual_ `<assert.h>` header file](https://github.com/bminor/glibc/blob/master/assert/assert.h).)

This header file reveals another important property of assertions – they can be _turned off_. In C and C++, defining the preprocessor variable `NDEBUG` ("no debug") turns off assertions, replacing them with a statement that does nothing. The `NDEBUG` variable can be set during compilation:

In [ ]:
!cc -DNDEBUG -o testassert testassert.c

And, as you can see, the assertion has no effect anymore:

In [ ]:
!./testassert

In Python, assertions can also be turned off, by invoking the `python` interpreter with the `-O` ("optimize") flag:

In [ ]:
!python -c 'assert 2 + 2 == 5; print("Foo")'

In [ ]:
!python -O -c 'assert 2 + 2 == 5; print("Foo")'

In comparison, which language wins in the amount of assertion diagnostics? Have a look at the information Python provides. If, after defining `fun()` as

In [ ]:
def fun():
    assert 2 + 2 == 5

In [ ]:
quiz("If we invoke fun() and the assertion fails,"
     " which information do we get?",
     [
         "The failing condition (2 + 2 == 5)",
         "The location of the assertion in the program",
         "The list of callers",
         "All of the above"
     ],
     123456789 % 5
    )

Indeed, a failed assertion (like any exception in Python) provides us with lots of debugging information, even including the source code:

In [ ]:
with ExpectError():
    fun()

## Checking Preconditions

Assertions show their true power when they are not used in a test, but used in a program instead, because that is when they can check not only _one_ run, but actually _all_ runs.

The classic example for the use of assertions is a _square root_ program, implementing the function $\sqrt{x}$.  (Let's assume for a moment that the environment does not already have one.) 

We want to ensure that `square_root()` is always called with correct arguments. For this purpose, we set up an assertion:

In [ ]:
def square_root(x):
    assert x >= 0
    ...   # compute square root in y

This assertion is called the _precondition_. A precondition is checked at the beginning of a function.  It checks whether all the conditions for using the function are met.  

So, if we call `square_root()` with an bad argument, we will get an exception. This holds for _any_ call, ever.

In [ ]:
with ExpectError():
    square_root(-1)

For a dynamically typed language like Python, an assertion could actually also check that the argument has the correct type. For `aquare_root()`, we could ensure that `x` actually has a numeric type:

In [ ]:
def square_root(x):
    assert isinstance(x, (int, float))
    assert x >= 0
    ...   # compute square root in y

And while calls with the correct types just work...

In [ ]:
square_root(4) 

In [ ]:
square_root(4.0)

... a call with an illegal type will raise a revealing diagnostic:

In [ ]:
with ExpectError():
    square_root('4')

In [ ]:
quiz("If we did not check for the type of x, "
     "would the assertion 'x >= 0' still catch a bad call?",
     [
         "Yes, since >= is only defined between numbers",
         "No, because an empty list or string would evaluate to 0"
     ],
     0b10 - 0b01
    )

Fortunately (for us Python users), the assertion `x >= 0` would already catch a number of invalid types, because (in contrast to, say, JavaScript), Python has no implicit conversion of strings or structures to integers:

In [ ]:
with ExpectError():
    '4' >= 0

## Checking Results

While a precondition ensures that the _argument_ to a function is correct, a _postcondition_ checks the _result_ of this very function (assuming the precondition held in the first place). For our `square_root()` function, we can check that the result $y = \sqrt{x}$ is correct by checking that $y^2 = x$ holds:

In [ ]:
def square_root(x):
    assert x >= 0
    ...   # compute square root in y
    assert y * y == x

In practice, we might encounter problems with this assertion. What might these be?

In [ ]:
quiz("Why could the assertion fail despite square_root() being correct?",
     [
         "We need to compute y^2, not y * y",
         "We may encounter rounding errors",
         "The value of x may have changed during computation",
         "The interpreter / compiler may be buggy"
     ],
    0b110011 - 0o61
    )

Technically speaking, there could be many things that _also_ could cause the assertion to fail (cosmic radiation, operating system bugs, secret service bugs, anything) – but the by far most important reason is indeed rounding errors. Here's a simple example, using the Python built-in square root function:

In [ ]:
import math

In [ ]:
math.sqrt(2.0) * math.sqrt(2.0)

In [ ]:
math.sqrt(2.0) * math.sqrt(2.0) == 2.0

If you want to compare two floating-point values, you need to provide an _epsilon value_ denoting the margin of error.

In [ ]:
def square_root(x):
    assert x >= 0
    ...   # compute square root in y
    epsilon = 0.000001
    assert abs(y * y - x) < epsilon

In Python, the function `math.isclose(x, y)` also does the job, by default ensuring that the two values are the same within about 9 decimal digits:

In [ ]:
math.isclose(math.sqrt(2.0) * math.sqrt(2.0), 2.0)

So let's use `math.isclose()` for our revised postcondition:

In [ ]:
def square_root(x):
    assert x >= 0
    ...   # compute square root in y
    assert math.isclose(y * y, x)

Let us try out this postcondition by using an actual implementation. The [Newton–Raphson method](https://en.wikipedia.org/wiki/Newton%27s_method) is an efficient way to compute square roots:

In [ ]:
def square_root(x):
    assert x >= 0  # precondition

    approx = None
    guess = x / 2
    while approx != guess:
        approx = guess
        guess = (approx + x / approx) / 2

    assert math.isclose(approx * approx, x)
    return approx

Apparently, this implementation does the job:

In [ ]:
square_root(4.0)

However, it is not just this call that produces the correct result – _all_ calls will produce the correct result. (If the postcondition assertion does not fail, that is.) So, a call like

In [ ]:
square_root(12345.0)

does not require us to _manually_ check the result – the postcondition assertion already has done that for us, and will continue to do so forever.

### Assertions and Tests

Having assertions right in the code gives us an easy means to _test_ it – if we can feed sufficiently many inputs into the code without the postcondition every failing, we can increase our confidence. Let us try this out with our `square_root()` function:

In [ ]:
for x in range(1, 10000):
    y = square_root(x)

Note again that we do not have to check the value of `y` – the `square_root()` postcondition already did that for us.

Instead of enumerating input values, we could also use random (non-negative) numbers; even totally random numbers could work if we filter out those tests where the precondition already fails. If you are interested in such _test generation techniques_, the [Fuzzing Book](fuzzingbook.org) is a great reference for you.

Modern program verification tools even can _prove_ that your program will always meet its assertions. But for all this, you need to have _explicit_ and _formal_ assertions in the first place.

For those interested in testing and verification, here is a quiz for you:

In [ ]:
quiz("Is there a value for x that satisfies the precondition, "
    "but fails the postcondition?",
     [ "Yes", "No" ],
     int("Y" in "Yes"))

This is indeed something a test generator or program verifier might be able to find with zero effort.

### Partial Checks

In the case of `square_root()`, our postcondition is _total_ – if it passes, then the result is correct (within the `epsilon` boundaries, that is). In practice, however, it is not always easy to provide such a total check. As an example, consider our `remove_html_markup()` function from the [Introduction to Debugging](Intro_Debug.ipynb):

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
remove_html_markup("I am a text with <strong>HTML markup</strong>")

The precondition for `remove_html_markup()` is trivial – it accepts any string. (Strictly speaking, a precondition `assert isinstance(s, str)` could prevent it from being called with some other collection such as a list.)

The challenge, however, is the _postcondition_. How do we check that `remove_html_markup()` produces the correct result?

* We could check it against some other implementation that removes HTML markup – but if we already do have such a "golden" implementation, why bother implementing it again?

* After a change, we could also check it against some earlier version to prevent _regression_ – that is, losing functionality that was there before. But how would we know the earlier version was correct? (And if it was, why change it?)

If we do not aim for ensuring full correctness, our postcondition can also check for _partial properties_. For instance, a postcondition for `remove_html_markup()` may simply ensure that the result no longer contains any markup:

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    # postcondition
    assert '<' not in out and '>' not in out

    return out

Besides doing a good job at checking results, the postcondition also does a good job in documenting what `remove_html_markup()` actually does.

### Assertions and Documentation

In contrast to "standard" documentation – "`square_root()` expects a non-negative number `x`; its result is $\sqrt{x}$" –, assertions have a big advantage: They are _formal_ – and thus have an unambiguous semantics. Notably, we can understand what a function does _uniquely by reading its pre- and postconditions_. Here is an example:

In [ ]:
def some_obscure_function(x, y, z):
    result = ...
    assert x == y == z or result > min(x, y, z)
    assert x == y == z or result < max(x, y, z)
    return result

In [ ]:
quiz("What does this function do?",
    [
        "It returns the minimum value out of x, y, z",
        "It returns the middle value out of x, y, z",
        "It returns the maximum value out of x, y, z",
    ],
      int(0.5 ** math.cos(math.pi))
    )

Indeed, this would be a useful (and bug-revaling!) postcondition for one of our showcase functions in the [chapter on statistical debugging](StatisticalDebugger.ipynb).

### Using Assertions to Trivially Locate Defects

The final benefit of assertions, and possibly even the most important in the context of this book, is _how much assertions help_ with locating defects.
Indeed, with proper assertions, it is almost trivial to locate the one function that is responsible for a failure.

Consider the following situation. Assume I have
* a function `f()` whose precondition is satisfied, calling
* a function `g()` whose precondition is violated and raises an exception.

In [ ]:
quiz("Which function is faulty here?",
     [
        "<code>g()</code> because it raises an exception",
        "<code>f()</code>"
        " because it violates the precondition of <code>g()</code>",
        "Both <code>f()</code> and <code>g()</code>"
        " because they are incompatible",
        "None of the above"
     ],
     math.factorial(int(math.tau / math.pi))
    )

The rule is very simple: If some function `func()` is called with its preconditions satisfied, and the postcondition of `func()` fail, then the fault in the program state must have originated at some point between these two events. Assuming that all functions called by `func()` also are correct (because their postconditions held), the defect _can only be in the code of `func()`._

What pre- and postconditions imply is actually often called a _contract_ between caller and callee:

* The caller promises to satisfy the _precondition_ of the callee, 
whereas
* the callee promises to satisfy its own _postcondition_, delivering a correct result.

In the above setting, `f()` is the caller, and `g()` is the callee; but as `f()` violates the precondition of `g()`, it has not kept its promises. Hence, `f()` violates the contract and is at fault. `f()` thus  needs to be fixed.

## Checking Data Structures

Let us get back to debugging.  In debugging, assertions serve two purposes: 

* They immediately detect bugs (if they fail)
* They immediately rule out _specific parts of code and state_ (if they pass)

This latter part is particularly interesting, as it allows us to focus our search on the lesser checked aspects of code and state. 

When we say "code _and_ state", what do we mean? Actually, assertions can not quickly check several executions of a function, but also _large amounts of data_, detecting faults in data _at the moment they are introduced_.

### Times and Time Bombs

Let us illustrate this by an example.  Let's assume we want a `Time` class that represents the time of day. Its constructor takes the current time using hours, minutes, and seconds. (Note that this is a deliberately simple example – real-world classes for representing time are way more complex.)

In [ ]:
class Time:
    def __init__(self, hours=0, minutes=0, seconds=0):
        self._hours = hours
        self._minutes = minutes
        self._seconds = seconds

To access the individual elements, we introduce a few getters:

In [ ]:
class Time(Time):
    def hours(self):
        return self._hours
    def minutes(self):
        return self._minutes
    def seconds(self):
        return self._seconds

We allow to print out time, using the [ISO 8601 format](https://en.wikipedia.org/wiki/ISO_8601):

In [ ]:
class Time(Time):
    def __repr__(self):
        return f"{self.hours():02}:{self.minutes():02}:{self.seconds():02}"

Three minutes to midnight can thus be represented as

In [ ]:
t = Time(23, 57, 0)
t

Unfortunately, there's nothing in our `Time` class that prevents blatant misuse.  We can easily set up a time with negative numbers, for instance:

In [ ]:
t = Time(-1, 0, 0)
t

Such a thing _may_ have some semantics (relative time. maybe?), but it's not exactly conforming to ISO format.

Even worse, we can even _construct_ a `Time` object with strings as numbers.

In [ ]:
t = Time("High noon")

and this will raise an error only at the moment we try to print it:

In [ ]:
with ExpectError():
    print(t)

Note how cryptic this error message must be for the poor soul who has to debug this: Where on earth do we have an `'=' alignment` in our code? The fact that the error comes from an _invalid value_ is totally lost.

In fact, what we have here is a _time bomb_ – a fault in the program state that can sleep for ages until someone steps on it. These are hard to debug, because one has to figure out when the time bomb was set – which can be thousands or millions of lines earlier in the program. Since in the absence of type checking, _any_ assignment to a `Time` object could be the culprit – so good luck with the search.

This is again where assertions save the day. What you need is an _assertion that checks whether the data is correct_. For instance, we could revise our constructor such that it checks for correct arguments:

In [ ]:
class Time(Time):
    def __init__(self, hours=0, minutes=0, seconds=0):
        assert 0 <= hours <= 23
        assert 0 <= minutes <= 59
        assert 0 <= seconds <= 60  # Includes leap seconds (ISO8601)

        self._hours = hours
        self._minutes = minutes
        self._seconds = seconds

These conditions check whether `hours`, `minutes`, and `seconds` are within the right range. They are called _data invariants_ (or short _invariants_) because they hold for the given data (notably, the internal attributes) at all times.

Note the unusual syntax for range checks (this is a Python special), and the fact that seconds can range from 0 to 60.  That's because there's not only leap years, but also leap seconds.

With this revised constructor, we now get errors as soon as we pass an invalid parameter:

In [ ]:
with ExpectError():
    t = Time(-23, 0, 0)

Hence, _any_ attempt to set _any_ Time object to an illegal state will be immediately detected. In other words, the time bomb defuses itself at the moment it is being set.

This means that when we are debugging, and search for potential faults in the state that could have caused the current failure, we can now rule out `Time` as a culprit, allowing us to focus on other parts of the state.

The more of the state we have checked with invariants,

* the less state we have to examine, 
* the fewer possible causes we have to investigate,
* the faster we are done with determining the defect.

### Invariant Checkers

For invariants to be effective, they have to be checked at all times. If we introduce a method that changes the state, then this method will also have to ensure that the invariant is satisfied:

In [ ]:
class Time(Time):
    def set_hours(self, hours):
        assert 0 <= hours <= 23
        self._hours = hours

If we have several methods that can alter an object, it can be helpful to factor out invariant checking into its own method.

### A repOK() Method for Time

By convention, methods that check invariants have the name `repOK()`, since they check whether the internal representation is okay, and return True if so.

Here's a `repOK()` method for `Time`:

In [ ]:
class Time(Time):
    def repOK(self):
        assert 0 <= self.hours() <= 23
        assert 0 <= self.minutes() <= 59
        assert 0 <= self.seconds() <= 60
        return True

We can integrate this method right into our constructor and our setter:

In [ ]:
class Time(Time):
    def __init__(self, hours=0, minutes=0, seconds=0):
        self._hours = hours
        self._minutes = minutes
        self._seconds = seconds
        assert self.repOK()

    def set_hours(self, hours):
        self._hours = hours
        assert self.repOK()

In [ ]:
with ExpectError():
    t = Time(-23, 0, 0)

Having a single method that checks everything can be beneficial, as it may explicitly check for more faulty states. For instance, it is still permissible to pass a floating-point number for hours and minutes, again breaking the Time representation:

In [ ]:
Time(12.5)

We can extend our `repOK()` method to check for correct types, too.

In [ ]:
class Time(Time):
    def repOK(self):
        assert isinstance(self.hours(), int)
        assert isinstance(self.minutes(), int)
        assert isinstance(self.seconds(), int)

        assert 0 <= self.hours() <= 23
        assert 0 <= self.minutes() <= 59
        assert 0 <= self.seconds() <= 60
        return True

In [ ]:
Time(14, 0, 0)

This now also catches other type errors:

In [ ]:
with ExpectError():
    t = Time("After midnight")

Our `repOK()` method can also be used in combination with pre- and postconditions. Typically, you'd like to make it part of the pre- and postcondition checks. Assume you want to implement an `advance()` method that adds a number of seconds to the current time.

In [ ]:
class Time(Time):
    def advance(self, seconds_offset):
        # Some complex computation
        ...

The preconditions and postconditions can be easily defined:

In [ ]:
class Time(Time):
    def seconds_since_midnight(self):
        return self.hours() * 3600 + self.minutes() * 60 + self.seconds()

    def advance(self, seconds_offset):
        old_seconds = self.seconds_since_midnight()
        ...  # Advance the clock
        assert (self.seconds_since_midnight() == 
                (old_seconds + seconds_offset) % (24 * 60 * 60))

But you'd really like `advance()` to check the state before and after its execution – again using `repOK()`:

In [ ]:
class Time(Time):
    def advance(self, seconds_offset):
        assert self.repOK()
        old_seconds = self.seconds_since_midnight()

        ...  # Advance the clock

        assert (self.seconds_since_midnight() == 
                (old_seconds + seconds_offset) % (24 * 60 * 60))
        assert self.repOK()

The first postcondition ensures that `advance()` produces the desired result; the second one ensures that the internal state is still okay.

### Large Data Structures

Invariants are especially useful if you have a large, complex data structure which is very hard to track in a conventional debugger.

Let's assume you have a [red-black tree](https://en.wikipedia.org/wiki/Red–black_tree) search tree for storing and searching data.  Red-black trees are among the most efficient data structures for representing associative arrays (also known as mappings); they are self-balancing and guarantee search, insertion, and deletion in logarithmic time. They also are among the most ugly to debug.

What is a red-black tree? Here is an example from Wikipedia:

![Red-Black Tree](https://upload.wikimedia.org/wikipedia/commons/6/66/Red-black_tree_example.svg)

As you can see, there are red nodes and black nodes (giving the tree its name). A red-black tree has a number of important properties that hold at all times – for instance, that the root node be black or that the tree be balanced. When we implement a red-black tree, these _invariants_ can be encoded into a `repOK()` method:

In [ ]:
class RedBlackTree:
    def repOK():
        assert self.rootHasNoParent()
        assert self.rootIsBlack()
        assert self.redNodesHaveOnlyBlackChildren()
        assert self.equalNumberOfBlackNodesOnSubtrees()
        assert self.treeIsAcyclic()
        assert self.parentsAreConsistent()
        return True

Each of these helper methods are checkers in their own right:

In [ ]:
class RedBlackTree(RedBlackTree):
    def rootHasNoParent(self):
        return self._root.parent is None
    def rootIsBlack(self):
        return self._root.color == self.BLACK
    ...

With all these helpers, our `repOK()` method will become very rigorous – but all this rigor is very much needed. Just for fun, check out the [description of red-black trees on Wikipedia](https://en.wikipedia.org/wiki/Red–black_tree). The description of how insertion or deletion work is 4 to 5 pages long (each!), with dozens of special cases that all have to be handled properly. If you ever face the task of implementing such a data structure, be sure to (1) write a `repOK()` method such as the above, and (2) call it before and after each method that alters the tree:

In [ ]:
class RedBlackTree(RedBlackTree):
    def insert(self, item):
        assert self.repOK()
        ...  # four pages of code
        assert self.repOK()

    def delete(self, item):
        assert self.repOK()
        ...  # five pages of code
        assert self.repOK()

Such checks will make your tree run much slower – essentially, instead of logarithmic time complexity, we now have linear time complexity, as the entire tree is traversed with each change – but you will find any bugs much, much faster. Once your tree goes in production, you can deactivate `repOK()` by default, using some debugging switch to turn it on again should the need ever arise:

In [ ]:
class RedBlackTree(RedBlackTree):
    def __init__(self, checkRepOK=False):
        ...
        self.checkRepOK = checkRepOK
    def repOK(self):
        if not self.checkRepOK:
            return True

        assert self.rootHasNoParent()
        assert self.rootIsBlack()
        ...

Just don't delete it – future maintainers of your code will be forever grateful that you have documented your assumptions and given them a means to quickly check their code.

### When Should Invariants be Checked?

During testing and debugging, invariants should be checked _at all times_, thus narrowing down the time it takes to detect a violation to a minimum. The easiest way to get there is to have them checked as _postcondition_ in the constructor and any other method that sets the state of an object.

If you have means to alter the state of an object outside of these methods – for instance, by directly writing to memory, or by writing to internal attributes –, then you may have to check them even more frequently. Using the [tracing infrastructure](Tracer.ipynb), for instance, you can have the tracer invoke `repOK()` with each and every line executed, thereby again directly pinpointing the moment the state gets corrupted. While this will slow down execution tremendously, it is still better to have the computer do the work than you stepping backwards and forwards through an execution.

Another question is whether assertions should remain active even in production code. Assertions take time, and this may be too much for production.

#### Assertions are not Production Code

First of all, assertions are _not_ production code – the rest of the code should not be impacted by any assertion being on or off. If you write code like

```python
assert map.remove(location)
```
your assertion will have a side effect, namely removing a location from the map.  If one turns assertions off, the side effect will be turned off as well.  You need to change this into

```python
locationRemoved = map.remove(location)
assert locationRemoved
```

#### For System Preconditions, Use Production Code 

Consequently, you should not rely on assertions for _system preconditions_  – that is, conditions that are necessary to keep the system running. System input (or anything that could be controlled by another party) still has to be validated by production code, not assertions. Critical conditions have to be checked by production code, not (only) assertions.

If you have code such as
```python
assert command in {"open", "close", "exit"}
exec(command)
```
then having the assertion document and check your assumptions is fine. However, if you turn the assertion off in production code, it will only be a matter of time until somebody enters a command like
```python
system("/bin/sh")
```
and all of a sudden takes control over your system.

#### Consider Leaving Some Assertions On

The main reason for turning assertions off is efficiency. However, _failing early is better than having bad data and not failing._ Think carefully which assertions have a high impact on execution time, and turn these off first. Assertions that have little to no impact on resources can be left on.

As an example, here's a piece of code that handles traffic in a simulation. The `light` variable can be either `RED`, `AMBER`, or `GREEN`:

```python
if light == RED:
   traffic.stop()
elif light == AMBER:
   traffic.prepare_to_stop()
elif light == GREEN:
   traffic.go()
else:
   pass   # This can't happen!
```

Having an assertion

```python
assert light in [RED. AMBER, GREEN]
```

in your code will eat some (minor) resources. However, adding a line

```python
assert False
```

in the place of the `This can't happen!` line, above, will still catch errors, but require no resources at all.

#### Define How Your Application Should Handle Internal Errors

By default, failing assertions are not exactly user-friendly – the diagnosis they provide is of interest to the code maintainers only.  Think of how your application should handle internal errors as discovered by assertions (or the runtime system). Simply exiting (as assertions on C do) may not be the best option for critical software. Think about implementing your own assert functions with appropriate recovery methods.

## System Invariants

In [ ]:
import cmath

In [ ]:
abs(cmath.exp(complex(0, -1) * cmath.pi).real)





## _Section 3_

\todo{Add}

_If you want to introduce code, it is helpful to state the most important functions, as in:_

* `random.randrange(start, end)` - return a random number [`start`, `end`]
* `range(start, end)` - create a list with integers from `start` to `end`.  Typically used in iterations.
* `for elem in list: body` executes `body` in a loop with `elem` taking each value from `list`.
* `for i in range(start, end): body` executes `body` in a loop with `i` from `start` to `end` - 1.
* `chr(n)` - return a character with ASCII code `n`

In [ ]:
import random

In [ ]:
def int_fuzzer():
    """A simple function that returns a random integer"""
    return random.randrange(1, 100) + 0.5

In [ ]:
# More code
pass

## _Section 4_

\todo{Add}

## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

In [ ]:
print(int_fuzzer())

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_